In [12]:
import pandas as pd
import numpy as np
import requests
import time
import re
import os
from request_simec import login, get_url, post_url
from restrictions import exemplares_aceitos, formatos_aceitos

In [13]:
df = pd.read_csv(
         'dados/TODOS.csv',
         dtype=str
    )

In [4]:
df.columns

Index(['row', 'id', 'Edital', 'CNPJ Editora', 'Editora', 'Código Coleção',
       'Coleção', 'Código Volume', 'Volume', 'Componente', 'Série', 'Formato',
       'Exemplar', 'Tipo de Arquivo', 'Arquivo', 'Header', 'Header get()',
       'Nome do Arquivo', 'Extensão do Arquivo', 'Deve ser baixado', 'Baixado',
       'Caminho Local', 'Extensão do Arquivo no Header'],
      dtype='object')

In [14]:
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36'}
proxies = {
 'http': 'http://192.168.0.105:3128',
 'https': 'http://192.168.0.105:3128',
}
s = requests.Session()
s.get(get_url)
s.post(post_url, data=login)

<Response [200]>

In [24]:
def login ():
    s = requests.Session()
    s.proxies = proxies
    s.get(get_url, proxies=proxies)
    s.post(post_url, data=login, proxies=proxies)
    
    return s

In [25]:
s = login()

TypeError: 'function' object is not iterable

In [16]:
i = 1
for index, row in df.iterrows():  
  if (row['Extensão do Arquivo'] in ['zip', 'x-zip-compressed']) and (row['Formato'] == 'Audiovisual') and (row['Deve ser baixado'] == 'Sim') and (row['Baixado'] == 'Não'):
    
    print(str(i) + ' - ' + row['id'] + ' ' + row['Nome do Arquivo'] + ' ' + row['Extensão do Arquivo'])
    print
    
    try:
        r = s.get(row['Arquivo'], headers=headers)
    except Exception as e:
        print(e)
        break
    
    
    filename = 'arquivos/' + row['Edital'] + '/' + row['id'] + '/' + row['Nome do Arquivo']
    print(r.headers)
    
    if "Content-Lenght" in r.headers.keys():
        print('DEU RUIM')
        break
    
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, "wb") as f:
        f.write(r.content)
    
    df.at[index, 'Baixado'] = 'Sim'
    
    
    print('BAIXADO!')
    
    i += 1
    
    df.to_csv('dados/TODOS.csv', index=False)
    
    time.sleep(15)
    s.get(get_url, proxies=proxies)    
    print('zzz\n')
    time.sleep(15)

!calc()
    

1 - 4735 AUDVISU_geografia_9_PNLD2020_18116_araribamaisgeografia.zip zip
{'Date': 'Fri, 19 Mar 2021 21:12:50 GMT', 'Server': 'Apache', 'X-Powered-By': 'PHP/5.5.32 ZendServer/8.5.3', 'Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Pragma': 'no-cache', 'Content-Disposition': 'attachment; filename=AUDVISU_geografia_9_PNLD2020_18116_araribamaisgeografia.zip', 'Content-Type': 'application/zip', 'Age': '1', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive'}
BAIXADO!
zzz

2 - 4742 AUDVISU_historia_6_PNLD2020_18109_araribamaishistoria.zip zip
{'Date': 'Fri, 19 Mar 2021 21:15:43 GMT', 'Server': 'Apache', 'X-Powered-By': 'PHP/5.5.32 ZendServer/8.5.3', 'Expires': 'Thu, 19 Nov 1981 08:52:00 GMT', 'Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Pragma': 'no-cache', 'Content-Disposition': 'attachment; filename=AUDVISU_historia_6_PNLD2020_18109_araribamaishistoria.zip', 'Conten

In [15]:
for index, row in df.iterrows():
  if (row['id'] in ['4735', '4742', '4794', '4805']):
    df.at[index, 'Baixado'] = 'Não'
    print(row['id'] + ' ' + row['Baixado'])

4735 Não
4742 Não
4794 Não
4805 Não


In [ ]:
df[]

In [25]:
for index, row in df.iterrows():
  if (row['Extensão do Arquivo'] == 'text/html; charset=ISO-8859-1') and (row['Deve ser baixado'] == 'Sim') and (int(row['id']) >= 6614):
    df.at[index, 'Baixado'] = 'Não'

!calc()

In [6]:
df.to_csv('dados/TODOS.csv', index = False)

In [3]:
#####################################################################################################
for index, row in df.iterrows():
  if (row['Formato'] in formatos_aceitos) and (row['Exemplar'] in exemplares_aceitos):
    df.at[index, 'Deve ser baixado'] = 'Sim'
    df.at[index, 'Baixado'] = 'Não'

    header = s.head(row['Arquivo'])
    
    df.at[index, 'Header'] = header.headers
    
    
    if "content-disposition" in header.headers.keys():
        content_disposition = header.headers['content-disposition']
        
        try:
            f_name = re.findall("filename=(.+)", content_disposition)[0]
        except IndexError:
            f_name = content_disposition
            
    else:
        f_name = "NO CONTENT-DISPOSITION"
        
    if "Content-Type" in header.headers.keys():
        content_type = header.headers['Content-Type']
        
        try:
            c_type = re.findall("application/(.+)", content_type)[0]
        except IndexError:
            c_type = content_type
        
    else:
        c_type = "NO CONTENT-TYPE"
        
    
    df.at[index, 'Nome do Arquivo'] = f_name
    df.at[index, 'Extensão do Arquivo'] = c_type
    
    print(row['id'] + ' ' + f_name + ' ' + c_type)

  else:
    df.at[index, 'Deve ser baixado'] = 'Não'
    print(row['id'] + ' ****** ' + 'NÃO DEVE SER BAIXADO')
    

NameError: name 'df' is not defined